# Практическое задание к уроку 5

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  

Using TensorFlow backend.


In [2]:
from sklearn.model_selection import train_test_split

In [3]:
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.1.0
Eager mode:  True
Hub version:  0.10.0
GPU is available


Задание из 2-х частей.
Берем отызывы за лето (из архива с материалами или предыдущего занятия)
1. Учим conv сеть для классификации - выбить auc выше 0.95

In [4]:
data=pd.read_excel('отзывы за лето.xls')
data.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [5]:
df_train=data.copy()
#df_train, df_val=train_test_split(data, test_size=0.33, random_state=42)
#df_train, df_val=train_test_split(train, test_size=0.2, random_state=42) 

In [6]:
num_classes=data['Rating'].value_counts().shape[0]
num_classes+=1

In [7]:
max_words = 200
max_len = 40

# Training
epochs = 20
batch_size =32
print_batch_n = 100

In [8]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['Content'] = df_train['Content'].apply(preprocess_text)
#df_val['Content'] = df_val['Content'].apply(preprocess_text)
#df_test['Content'] = df_test['Content'].apply(preprocess_text)

In [9]:
train_corpus = " ".join(df_train["Content"])
train_corpus = train_corpus.lower()

In [10]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Cats\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
tokens_filtered = [word for word in tokens if word.isalnum()]

dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [12]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [13]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [14]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["Content"]], dtype=np.int32)
#x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["Content"]], dtype=np.int32)
#x_val = np.asarray([text_to_sequence(text, max_len) for text in df_val["Content"]], dtype=np.int32)

In [15]:

y_train = keras.utils.to_categorical(df_train["Rating"], num_classes)
#y_val = keras.utils.to_categorical(df_val["Rating"], num_classes)

In [284]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
#model.add(Dropout(0.5))
model.add(Conv1D(1024, 7))
model.add(Activation("relu"))
#model.add(Dropout(0.5))
model.add(Conv1D(1024, 7))
model.add(Activation("relu"))


model.add(GlobalMaxPool1D())

model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [285]:
model.summary()

Model: "sequential_64"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_64 (Embedding)     (None, 40, 128)           25600     
_________________________________________________________________
conv1d_123 (Conv1D)          (None, 34, 1024)          918528    
_________________________________________________________________
activation_213 (Activation)  (None, 34, 1024)          0         
_________________________________________________________________
conv1d_124 (Conv1D)          (None, 28, 1024)          7341056   
_________________________________________________________________
activation_214 (Activation)  (None, 28, 1024)          0         
_________________________________________________________________
global_max_pooling1d_61 (Glo (None, 1024)              0         
_________________________________________________________________
dense_105 (Dense)            (None, 10)              

In [286]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [287]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=20,
                    verbose=1,
                    validation_split=0.1,
                    )

Train on 18593 samples, validate on 2066 samples
Epoch 1/20
18593/18593 [==============================] - 38s 2ms/step - loss: 0.2088 - accuracy: 0.9259 - val_loss: 0.2311 - val_accuracy: 0.9095
Epoch 2/20
18593/18593 [==============================] - 37s 2ms/step - loss: 0.1875 - accuracy: 0.9315 - val_loss: 0.2219 - val_accuracy: 0.9163
Epoch 3/20
18593/18593 [==============================] - 37s 2ms/step - loss: 0.1834 - accuracy: 0.9326 - val_loss: 0.2281 - val_accuracy: 0.9116
Epoch 4/20
18593/18593 [==============================] - 37s 2ms/step - loss: 0.1805 - accuracy: 0.9345 - val_loss: 0.2297 - val_accuracy: 0.9149
Epoch 5/20
18593/18593 [==============================] - 37s 2ms/step - loss: 0.1752 - accuracy: 0.9361 - val_loss: 0.2341 - val_accuracy: 0.9112
Epoch 6/20
18593/18593 [==============================] - 37s 2ms/step - loss: 0.1706 - accuracy: 0.9383 - val_loss: 0.2376 - val_accuracy: 0.9144
Epoch 7/20
18593/18593 [==============================] - 38s 2ms/ste

 38s 2ms/step - loss: 0.1221 - accuracy: 0.9561  модель немного переучилась но accuracy: 0.9561

#### 2. Берём предобученный word2vec и его эмбедингами инициализируем сетку, как влияет на качество?

In [16]:
import gensim

In [19]:
word2Vec=gensim.models.Word2Vec(size=200, window=5, min_count=2, sg = 1, hs = 0, negative = 10, workers= 32, seed = 34)

In [20]:
word2Vec.build_vocab(df_train['Content'])

In [21]:
word2Vec.train(df_train['Content'],total_examples=df_train['Content'].shape[0],epochs=20) 

(3570870, 17175380)

In [22]:
model = Sequential()
model.add(word2Vec.wv.get_keras_embedding(train_embeddings=False))
#model.add(Dropout(0.5))
model.add(Conv1D(128, 7))
model.add(Activation("relu"))
#model.add(Dropout(0.5))
model.add(Conv1D(128, 7))
model.add(Activation("relu"))


model.add(GlobalMaxPool1D())

model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [23]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [24]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=20,
                    verbose=1,
                    validation_split=0.1,
                    )

Train on 18593 samples, validate on 2066 samples
Epoch 1/20
18593/18593 [==============================] - 8s 417us/step - loss: 0.2356 - accuracy: 0.9192 - val_loss: 0.2586 - val_accuracy: 0.9013
Epoch 2/20
18593/18593 [==============================] - 5s 277us/step - loss: 0.2078 - accuracy: 0.9254 - val_loss: 0.2592 - val_accuracy: 0.9008
Epoch 3/20
18593/18593 [==============================] - 5s 282us/step - loss: 0.2018 - accuracy: 0.9268 - val_loss: 0.2555 - val_accuracy: 0.9025
Epoch 4/20
18593/18593 [==============================] - 5s 281us/step - loss: 0.1993 - accuracy: 0.9269 - val_loss: 0.2505 - val_accuracy: 0.9034
Epoch 5/20
18593/18593 [==============================] - 5s 281us/step - loss: 0.1957 - accuracy: 0.9282 - val_loss: 0.2415 - val_accuracy: 0.9038
Epoch 6/20
18593/18593 [==============================] - 5s 279us/step - loss: 0.1928 - accuracy: 0.9288 - val_loss: 0.2365 - val_accuracy: 0.9055
Epoch 7/20
18593/18593 [==============================] - 5s 28

предобученый эмбедингами позволяет сократить аохитектуру сети без особой потери качества